In [6]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from focal_loss import BinaryFocalLoss
from tensorflow.keras.optimizers.legacy import Adam


In [7]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=1,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [9]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use linear activation for Hinge loss
])


In [21]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])


In [23]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 50s 35ms/step - loss: 0.5388 - accuracy: 0.7477 - val_loss: 0.4289 - val_accuracy: 0.7743
Epoch 2/20
1399/1399 [==============================] - 51s 37ms/step - loss: 0.3860 - accuracy: 0.8134 - val_loss: 0.2699 - val_accuracy: 0.8472
Epoch 3/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.3061 - accuracy: 0.8606 - val_loss: 0.2864 - val_accuracy: 0.8507
Epoch 4/20
1399/1399 [==============================] - 67s 48ms/step - loss: 0.2408 - accuracy: 0.8928 - val_loss: 0.2387 - val_accuracy: 0.8924
Epoch 5/20
1399/1399 [==============================] - 55s 40ms/step - loss: 0.2147 - accuracy: 0.9099 - val_loss: 0.1803 - val_accuracy: 0.9097
Epoch 6/20
1399/1399 [==============================] - 75s 53ms/step - loss: 0.1740 - accuracy: 0.9192 - val_loss: 0.1402 - val_accuracy: 0.9236
Epoch 7/20
1399/1399 [==============================] - 95s 68ms/step - loss: 0.1464 - accuracy: 0.9407 - val_loss: 0.1141 -

In [24]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 154ms/step - loss: 0.0382 - accuracy: 0.9896
Test Loss: 0.03820107877254486, Test Accuracy: 0.9895833134651184


In [10]:
from tensorflow.keras.losses import BinaryFocalCrossentropy

In [11]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryFocalCrossentropy(), metrics=['accuracy'])


In [12]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
1399/1399 [==============================] - 76s 54ms/step - loss: 0.1396 - accuracy: 0.7370 - val_loss: 0.1276 - val_accuracy: 0.7535
Epoch 2/20
1399/1399 [==============================] - 55s 39ms/step - loss: 0.1042 - accuracy: 0.8213 - val_loss: 0.0819 - val_accuracy: 0.8472
Epoch 3/20
1399/1399 [==============================] - 79s 56ms/step - loss: 0.0777 - accuracy: 0.8585 - val_loss: 0.0532 - val_accuracy: 0.8889
Epoch 4/20
1399/1399 [==============================] - 86s 61ms/step - loss: 0.0630 - accuracy: 0.8942 - val_loss: 0.0499 - val_accuracy: 0.9132
Epoch 5/20
1399/1399 [==============================] - 87s 62ms/step - loss: 0.0479 - accuracy: 0.9171 - val_loss: 0.0362 - val_accuracy: 0.9271
Epoch 6/20
1399/1399 [==============================] - 87s 62ms/step - loss: 0.0405 - accuracy: 0.9357 - val_loss: 0.0261 - val_accuracy: 0.9549
Epoch 7/20
1399/1399 [==============================] - 86s 62ms/step - loss: 0.0326 - accuracy: 0.9543 - val_loss: 0.0210 -

In [13]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 3s 282ms/step - loss: 0.0492 - accuracy: 0.9618
Test Loss: 0.049241337925195694, Test Accuracy: 0.9618055820465088
